In [1]:
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
warnings.filterwarnings('ignore')
import pymorphy2

In [2]:
df = pd.read_csv('df.csv', index_col=0)

# 4.1 Разработка бота

## Функции бота

In [3]:
def cleaning(words):
    words = re.sub(r'[^a-Za-za-яА-Я]', ' ', str(words)).lower().strip()
    stopwords =  ["ул", "д", "согласно", "c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля", "здравствуйте","знать","значит","значить","й","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","маленькмй", "мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]
    words = ' '.join([word for word in words.split() if word not in (stopwords)])
    morph = pymorphy2.MorphAnalyzer()
    words = ' '.join(morph.parse(word)[0].normal_form for word in words.split())
    return words

In [4]:
def load_model(filename = 'modell.pkl'):
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    return model

In [5]:
model = load_model()

In [6]:
model

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(penalty='none', solver='saga'))])

# Бот с параметром жесткости

In [7]:
def make_pred(words, model, p_num=1):
    if p_num == 1:
        pred = model.predict([words])
        return pred
    else:
        allpreds = model.predict_proba([words]).reshape(-1)
        R=[]
        for i in allpreds.argsort()[-p_num:][::-1]:
            R.append(model.classes_[i])
        return R

In [8]:
def helpp():
    print("Bot: Если у вас возникли какие-либо проблемы, убедитесь что все ваши файлы находятся в одной директории и вы везде указываете расширение файлов. В директрории должен обязательно содержаться файл model.pkl. Если вы считаете что вам выдается неточный результат знайте, что чем больше вы расскажете или напишите в вашем резюме о себе, тeм точнее будет результат. Eсли у вас есть обученная модель в формате pkl то вы можете заменить файл model.pkl в директории на свой \n\nВ боте содержaтся следующие функции load_model(filename = 'model.pkl'),pdf_text(filename), cleaning(words), make_pred(words, model, p_num=1), helpp()")

In [11]:
print("Loading modules...")
#<=============IMPORTS=============>
import pdfplumber
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
import pickle
import re
import numpy as np
import pymorphy2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#<=============Functions=============>
#loading model
def load_model(filename = "model.pkl"):
    with open(filename, 'rb') as file:
            model = pickle.load(file)
    return model

#predict
def make_pred(words, model, p_num=1):
    if p_num==1:
        pred = model.predict([words])
        return pred
    else:
        R = []
        # выбор нескольких алресатов
        allpreds = model.predict_proba([words]).reshape(-1) 
        for i in allpreds.argsort()[-p_num:][::-1]:
            R.append(model.classes_[i])
        return R
    
#words cleaning
def cleaning(words):
    print(">>> Всего слов: ", len(words.split()))
    morph = pymorphy2.MorphAnalyzer()
    a=len(words.split()) 
    words = re.sub('[^a-zA-zа-яА-Я]', ' ', words).lower().strip()
    stopwords = ["тд","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]
    words = ' '.join([word for word in words.split() if word not in (stopwords)])
    words = ' '.join([morph.parse(w)[0].normal_form for w in words.split()])
    print(">>> Убрано: ", a - len(words.split()))
    return words


# help
def helpp():
    print("Bot: Если у вас возникли какие-либо проблемы, убедитесь что все ваши файлы находятся в одной директории и вы везде указываете расширение файлов. В директрории должен обязательно содержаться файл model.pkl. Если вы считаете что вам выдается неточный результат знайте, что чем больше вы расскажете или напишите в вашем резюме о себе, тeм точнее будет результат. Eсли у вас есть обученная модель в формате pkl то вы можете заменить файл model.pkl в директории на свой \n\nВ боте содержaтся следующие функции load_model(filename = 'model.pkl'),pdf_text(filename), cleaning(words), make_pred(words, model, p_num=1), helpp()")
#<=============Functions=============>

print("Done")
print()



def main():
    while True:
        a = input("Bot: Для получения справки введите 'help' или 'no' если она вам не нужна \nUser: ")
        if a == "help":
            helpp()

        while True:
            a = input("Bot: Введите имя pdf файла с РАСШИРЕНИЕМ или напишите содержания письма/документа \nUser: ")
            if "pdf" in a:
                try:
                    text = pdf_text(a)
                    break
                except Exception:
                    print("Bot: Что то не так с файлом ")
            else:
                print("Bot: Вы ввели: ", a)
                text = a
                break
        text = cleaning(text)
        print("Bot: loading model")
        model = load_model(filename="model.pkl")
        n = int(input("Bot: Сколько вы хотите получить адресатов: \nUser: "))
        R = make_pred(text, model, p_num=n)
        print("<========RESULT========>")
        for i in R:
            print(i)
        print("<========RESULT========>")
        
                            
                    
        a = input("Bot: Хотите повторить?(y/n) \nUser: ")
        if a == "y":
            print("<====================================================>")
            continue
        else:
            break

main()



Loading modules...
Done

Bot: Для получения справки введите 'help' или 'no' если она вам не нужна 
User: no
Bot: Введите имя pdf файла с РАСШИРЕНИЕМ или опишите то что вы умеете как можно подробнее 
User: кому бы можно было направить письмо, в котором написано об участии в чемпионате профессионального мастерства
Bot: Вы ввели:  кому бы можно было направить письмо, в котором написано об участии в чемпионате профессионального мастерства
>>> Всего слов:  15
>>> Убрано:  7
Bot: loading model
Bot: Сколько вы хотите получить предсказанных компетенций: 
User: 3
<========RESULT========>
Уразов Р.Н.
Гонтарь М.А.
Иванюк Л.А.
<========RESULT========>
Bot: Хотите узнать блок хз что писать(y/n) 
User: y
Bot: Хотите повторить?(y/n) 
User: n
